- Welche Features sollen verwendet werden?
- Standardisierung / Skalierung wie?
- Ziel der Clusteranalyse formulieren

Inhaltsübersicht

noch machen!!!

In [ ]:
# imports
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline

In [ ]:
df = pd.read_csv("data/cleaned_air_quality_data_2025-03-27.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# Liste der Schadstoffspalten
pollutants = ['Co', 'No2', 'O3', 'Pm10', 'Pm25', 'So2']

# Anteil fehlender Werte pro Spalte
missing_ratios = df[pollutants].isna().mean().sort_values(ascending=True)

print("Anteil fehlender Werte pro Schadstoff:")
print(missing_ratios)


In [ ]:
# Gruppieren: Für jede Stadt den Mittelwert je Schadstoff berechnen
city_pollution_avg = df.groupby('City')[pollutants].mean()

# Ergebnis prüfen
print(city_pollution_avg.head())


In [ ]:
# Städte mit vollständigen Werten (alle 6 Schadstoffe nicht NaN)
city_pollution_complete = city_pollution_avg.dropna()
print(f"Anzahl Städte mit vollständigen Daten: {city_pollution_complete.shape[0]}")


In [ ]:

# Mittelwerte für Koordinaten je Stadt berechnen
coords = df.groupby('City')[['Latitude', 'Longitude']].mean()

# Nur die Städte mit vollständigen Schadstoffdaten behalten
coords_filtered = coords.loc[city_pollution_complete.index]



In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# GeoDataFrame mit Punkt-Geometrie
coords_filtered['geometry'] = coords_filtered.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
gdf_complete = gpd.GeoDataFrame(coords_filtered, geometry='geometry', crs='EPSG:4326')


In [ ]:
import geodatasets

# Weltkarte laden
world = gpd.read_file(geodatasets.get_path('naturalearth.land'))

# Plot
ax = world.plot(figsize=(12, 6), color='lightgrey', edgecolor='white')
gdf_complete.plot(ax=ax, color='green', markersize=5)

ax.set_title("Städte mit vollständigen Daten für alle sechs Schadstoffe")


In [ ]:
# Daten skalieren, damit die Einheiten der Schadstoffe vergleichbar sind


from sklearn.preprocessing import StandardScaler

# DataFrame mit Schadstoffwerten: city_pollution_complete
# → enthält nur Städte mit vollständigen Werten

# Sicherstellen, dass nur die Werte (nicht die Geo-Spalten) verwendet werden
X = city_pollution_complete.copy()

# Normalisieren
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Optional: In DataFrame umwandeln für bessere Lesbarkeit
import pandas as pd
X_scaled_df = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)

print(X_scaled_df.head())


In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Für verschiedene Clusterzahlen die "Inertia" berechnen
inertias = []
k_range = range(1, 31)  # z. B. 1 bis 10 Cluster

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# Plot der Elbow-Kurve
plt.figure(figsize=(8, 5))
plt.plot(k_range, inertias, marker='o')
plt.xlabel('Anzahl der Cluster (k)')
plt.ylabel('Inertia')
plt.title('Elbow-Methode zur Bestimmung der optimalen Clusteranzahl')
plt.grid(True)
plt.show()


Elbow-Methode nicht eindeutig; es gibt aber ziemlich sicher noch echte Ausreißerstädte, also nehmen wir jetz terst mal 6 Cluster und sehen dann weiter.

In [ ]:
# Clustering mit 6 Clustern
kmeans = KMeans(n_clusters=6, random_state=42)
labels = kmeans.fit_predict(X_scaled)

# Clusterlabels zum ursprünglichen DataFrame hinzufügen
city_pollution_complete = city_pollution_avg.dropna().copy()
city_pollution_complete['Cluster'] = labels

# Vorschau: Welche Stadt gehört zu welchem Cluster?
print(city_pollution_complete[['Cluster']].value_counts().sort_index())


In [ ]:
# Beispiel: Cluster 3 hat nur eine Stadt
einzel_clusters = city_pollution_complete['Cluster'].value_counts()
einzel_clusters = einzel_clusters[einzel_clusters == 1].index.tolist()

# Städte aus diesen Clustern anzeigen
einzelstaedte = city_pollution_complete[city_pollution_complete['Cluster'].isin(einzel_clusters)]

print(einzelstaedte.index.tolist())  # Stadt-Namen


In [ ]:
print(einzelstaedte)

## Städte mit extremen Schadstoffprofilen

### Temuco (Chile)
Temuco wurde als Extremfall von der Clusteranalyse ausgeschlossen, da die Stadt durch außergewöhnlich hohe Schwefeldioxid-Werte (SO₂ ≈ 85.6 µg/m³) auffiel, die in keiner anderen Stadt annähernd erreicht wurden. Diese Belastung ist vermutlich auf die Kombination aus häufiger Holzverbrennung im Winter und der Nähe zum aktiven Vulkan Villarrica zurückzuführen. Auch die PM2.5-Werte lagen mit 67.5 µg/m³ deutlich über dem Mittel.

Aufgrund dieses sehr speziellen Luftschadstoffprofils hätte Temuco ein eigenes Cluster gebildet und dadurch die Vergleichbarkeit der übrigen Gruppen verzerrt. Die Stadt wurde daher in der Hauptanalyse ausgeschlossen

### Ashkelon (Israel)
Ashkelon wurde aufgrund technischer Auffälligkeiten ausgeschlossen. Der CO-Wert liegt mit durchschnittlich 138.7 µg/m³ mehr als eine Größenordnung über allen anderen Städten im Datensatz. Auch die übrigen Schadstoffwerte wirken inkonsistent. Die stark abweichenden Werte deuten auf eine defekte oder fehlerhafte CO-Messstation hin.

Um eine Verzerrung der Clusteranalyse durch fehlerhafte Daten zu vermeiden, wurde Ashkelon entfernt. Der Ausschluss erfolgte nicht, um extreme, aber plausible Umweltbedingungen auszublenden, sondern auf Basis der Datenqualität.

In [ ]:
# Liste der Städte, die aus der Hauptanalyse entfernt werden:
ausreisser = ['Ashkelon', 'Temuco']

# Neue Version des DataFrames ohne diese beiden
city_pollution_cleaned = city_pollution_complete.drop(index=ausreisser)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
city_pollution_scaled_cleaned = scaler.fit_transform(city_pollution_cleaned)

In [ ]:
from sklearn.cluster import KMeans

kmeans_cleaned = KMeans(n_clusters=6, random_state=42)
labels_cleaned = kmeans_cleaned.fit_predict(city_pollution_scaled_cleaned)

# Clusterlabels zum DataFrame hinzufügen
city_pollution_cleaned['Cluster'] = labels_cleaned


In [ ]:
city_pollution_cleaned['Cluster'].value_counts().sort_index()


In [ ]:
# Beispiel: Cluster 3 hat nur eine Stadt
einzel_clusters = city_pollution_cleaned['Cluster'].value_counts()
einzel_clusters = einzel_clusters[einzel_clusters == 1].index.tolist()

# Städte aus diesen Clustern anzeigen
einzelstaedte = city_pollution_cleaned[city_pollution_cleaned['Cluster'].isin(einzel_clusters)]

print(einzelstaedte.index.tolist())  # Stadt-Namen

In [ ]:
print(einzelstaedte)

### Khorramshahr (Iran)

Khorramshahr wurde aufgrund seines außergewöhnlich hohen Ozon- und Feinstaubniveaus aus der Hauptclusteranalyse ausgeschlossen. Die Werte deuten nicht auf Messfehler, sondern auf eine ernsthafte Luftbelastung hin, die möglicherweise durch regionale Industrieaktivität, hohe Temperaturen und Photochemie verstärkt wird.

Die Stadt bildet aufgrund ihres Extremprofils ein eigenes Cluster, wodurch die übrige Gruppierung verzerrt würde. Daher wurde sie separat dokumentiert, aber nicht in die finale Clusterstruktur aufgenommen.

In [ ]:
# Liste aktualisieren
ausreisser = ['Ashkelon', 'Temuco', 'Khorramshahr']

# Gefilterter DataFrame
X_cleaned_final = city_pollution_complete.drop(index=ausreisser)

In [ ]:
scaler = StandardScaler()
X_scaled_final = scaler.fit_transform(X_cleaned_final)


In [ ]:
kmeans_final = KMeans(n_clusters=6, random_state=42)
labels_final = kmeans_final.fit_predict(X_scaled_final)

# Clusterlabels hinzufügen
X_cleaned_final['Cluster'] = labels_final


In [ ]:
X_cleaned_final['Cluster'].value_counts().sort_index()

## Erklärung für Wiebke, warum jetzt PCA
Wir haben sechs Schadstoffe, müssen die aber zweidimensional plotten. Deshalb reduzieren wir die sechs Dimensionen mit einer PCA auf zwei, also X- und Y-Achse. Das hat jetzt nichts mit geografischer Verteilung zu tun, das zeigt nur, wie nah welche Cluster beieinander liegen.

In [ ]:
from sklearn.decomposition import PCA

# PCA auf die skalierten Daten anwenden (X_scaled_final aus dem letzten Schritt)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled_final)

# In DataFrame mit Clustern zusammenführen
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'], index=X_cleaned_final.index)
pca_df['Cluster'] = X_cleaned_final['Cluster']

# Farben definieren (automatisch, keine Sorge 😉)
plt.figure(figsize=(10, 6))
for cluster in sorted(pca_df['Cluster'].unique()):
    subset = pca_df[pca_df['Cluster'] == cluster]
    plt.scatter(subset['PC1'], subset['PC2'], label=f'Cluster {cluster}', s=50, alpha=0.7)

plt.title("PCA-Visualisierung der Cluster (2D)")
plt.xlabel("Hauptkomponente 1")
plt.ylabel("Hauptkomponente 2")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## 📍 Beschreibung der Clusterstruktur (PCA-Visualisierung)

Die nachfolgende PCA-Visualisierung zeigt die Verteilung der Städte im zweidimensionalen Raum, basierend auf ihren durchschnittlichen Schadstoffwerten (CO, NO₂, O₃, PM10, PM2.5, SO₂). Die ursprünglichen sechs Dimensionen wurden mithilfe einer Hauptkomponentenanalyse (PCA) auf zwei Hauptachsen reduziert, die möglichst viel der Varianz im Datensatz abbilden. Die Punkte im Plot repräsentieren einzelne Städte, eingefärbt nach ihrer jeweiligen Clusterzugehörigkeit (KMeans, k=6).

In der linken Hälfte des Plots gruppieren sich die Cluster 1, 3, 4 und 5:

- Cluster 5 erscheint besonders kompakt und dicht, was auf ein sehr homogenes Schadstoffprofil der zugehörigen Städte hindeutet.

- Cluster 1 und Cluster 3 liegen in direkter Nachbarschaft zu Cluster 5 und sind ebenfalls gut erkennbar voneinander abgegrenzt, jedoch weniger dicht. In einem kleinen Bereich überschneiden sich die drei Cluster leicht, was auf gewisse Ähnlichkeiten in den Profilen hindeutet.

- Cluster 4 liegt ebenfalls in der linken Plot-Hälfte, ist jedoch vollständig von den anderen Gruppen abgegrenzt. Die Punktwolke ist weniger dicht, aber klar umrissen, was auf größere interne Varianz, jedoch gute Abgrenzbarkeit hindeutet.

In der rechten Hälfte befinden sich Cluster 0 und Cluster 2:

- Cluster 0 bildet eine kompakte, klar erkennbare Gruppe – möglicherweise mit einem sehr spezifischen Schadstoffmuster.

- Cluster 2 ist das am stärksten gestreute Cluster und wirkt visuell weniger zusammenhängend. Ein einzelner Punkt liegt sogar deutlich entfernt in der linken Plothälfte, nahe bei Cluster 3. Die Zugehörigkeit zu Cluster 2 wird hier nur durch die algorithmische Einfärbung deutlich.

Die Clusterstruktur ist insgesamt gut differenziert und bietet eine fundierte Grundlage für die inhaltliche Analyse der Luftverschmutzungsprofile.

In [ ]:
# Mittelwert pro Schadstoff je Cluster

# Mittelwerte der Schadstoffe je Cluster
cluster_profiles = X_cleaned_final.groupby('Cluster')[['Co', 'No2', 'O3', 'Pm10', 'Pm25', 'So2']].mean()

# Optional: auf 2 Dezimalstellen runden für bessere Lesbarkeit
cluster_profiles_rounded = cluster_profiles.round(2)

# Anzeigen
print(cluster_profiles_rounded)


In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Koordinaten zuordnen
coords = df.groupby('City')[['Latitude', 'Longitude']].mean()

# Nur die bereinigten Städte (ohne Ashkelon, Temuco, Khorramshahr)
coords_final = coords.loc[X_cleaned_final.index]

# GeoDataFrame bauen
coords_final['Cluster'] = X_cleaned_final['Cluster']
coords_final['geometry'] = coords_final.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
gdf_final = gpd.GeoDataFrame(coords_final, geometry='geometry', crs='EPSG:4326')


In [ ]:
import matplotlib.patches as mpatches

# Farbdefinition bleibt wie oben
cluster_colors = {
    1: '#4CAF50',  # mittelgrün
    5: '#1B5E20',  # dunkelgrün
    3: '#2196F3',  # mittelblau
    4: '#0D47A1',  # dunkelblau
    0: '#F7A1A1',  # hellrot
    2: '#B71C1C'   # dunkelrot
}

# Manuelle Gruppierung
legend_entries = [
    mpatches.Patch(color='#4CAF50', label='Cluster 1 – Niedrige Belastung (günstiges Klima)'),
    mpatches.Patch(color='#1B5E20', label='Cluster 5 – Niedrige Belastung (urbane Bedingungen)'),
    mpatches.Patch(color='#2196F3', label='Cluster 3 – Mittlere Belastung (Feinstaub)'),
    mpatches.Patch(color='#0D47A1', label='Cluster 4 – Mittlere Belastung (Ozon)'),
    mpatches.Patch(color='#F7A1A1', label='Cluster 0 – Starke Belastung (Feinstaub)'),
    mpatches.Patch(color='#B71C1C', label='Cluster 2 – Starke Belastung (CO und SO₂)')
]


In [ ]:
import geodatasets

# Weltkarte
world = gpd.read_file(geodatasets.get_path('naturalearth.land'))

# Plot
fig, ax = plt.subplots(figsize=(14, 8))
world.plot(ax=ax, color='lightgrey', edgecolor='white')

# Punkte pro Cluster plotten
for cluster in sorted(gdf_final['Cluster'].unique()):
    gdf_final[gdf_final['Cluster'] == cluster].plot(
        ax=ax,
        color=cluster_colors[cluster],
        markersize=20,
        label=f'Cluster {cluster}'
    )

# Benutzerdefinierte Legende
ax.legend(handles=legend_entries, title="Clustergruppen", loc='lower left', fontsize=10, title_fontsize=12)

# Titel & Layout
ax.set_title("Städte nach Luftverschmutzungs-Clustern (k=6)", fontsize=15)
ax.set_axis_off()
plt.tight_layout()
plt.show()


Die Clusteranalyse ergibt sechs klar unterscheidbare Luftverschmutzungsprofile:

- Cluster 1 (mittelgrün) und 5 (dunkelgrün) zeigen relativ geringe Schadstoffbelastungen
- Cluster 0 (rosa) und 2 (rot) stehen für stark belastete Städte – insbesondere mit hohen Feinstaub-, Stickstoff- und Kohlenmonoxidwerten.
- Cluster 3 (hellblau) und 4 (dunkelblau) nehmen Zwischenpositionen ein, mit jeweils spezifischen Merkmalen: Feinstaubdominanz (Cluster 3) bzw. erhöhte Ozonbelastung (Cluster 4)

Diese Differenzierung erlaubt eine strukturierte Einordnung der Städte nach ihrem Schadstoffprofil und bildet eine solide Grundlage für weiterführende Analysen oder politische Maßnahmen.

### Unterscheidung Cluster 1 und 5 (beide relativ saubere Luft)

Cluster 1 und Cluster 5 umfassen beide Städte mit insgesamt niedriger Luftverschmutzung. Dennoch unterscheidet der Algorithmus diese Gruppen aufgrund unterschiedlicher Schadstoffprofile, insbesondere bei Ozon (O₃) und Feinstaub (PM2.5).

Cluster 1 enthält vor allem Städte in Australien, Neuseeland, Kanada, Island und kleineren US-Regionen, oft in klimatisch günstigen, gut belüfteten oder dünn besiedelten Gegenden. Hier sind sowohl Ozon- als auch Feinstaubwerte durchgängig sehr niedrig. Die geringe Ozonbelastung ist besonders auffällig, da man intuitiv in Regionen wie Australien aufgrund des „Ozonlochs“ hohe Ozonwerte erwarten könnte. Tatsächlich bezieht sich das Ozonloch jedoch auf die Stratosphäre – bodennahes Ozon, das als Luftschadstoff wirkt, ist in diesen Städten gering.

Cluster 5 dagegen umfasst eine große Gruppe urbaner Zentren in Ländern wie Japan, Spanien, Frankreich, Deutschland, den USA und Großbritannien. Die Luftqualität ist hier weiterhin vergleichsweise gut, allerdings zeigen sich moderat erhöhte PM2.5- und Ozonwerte. Diese Belastungen lassen sich durch höhere Urbanisierung, Verkehrsdichte, sowie Photochemie in sonnenreichen Regionen erklären – insbesondere bei den Ozonwerten in südeuropäischen und japanischen Städten.

Die Trennung in zwei Cluster innerhalb der „sauberen Städte“ ist daher sachlich sinnvoll und differenzierend:
Cluster 1 steht für „sehr saubere Luft in klimatisch und strukturell begünstigten Regionen“,
Cluster 5 für „gute Luftqualität unter urbanen Rahmenbedingungen“.

In [ ]:
# Schadstoffmittelwerte der beiden Cluster extrahieren
cluster_1 = cluster_profiles.loc[1]
cluster_5 = cluster_profiles.loc[5]

# Differenz berechnen
diff = (cluster_5 - cluster_1).round(2)
print(diff)

In [ ]:
# Alle Städte aus Cluster 1
cluster_1_staedte = X_cleaned_final[X_cleaned_final['Cluster'] == 1].index

# Daten aus Original-DataFrame holen (nur eindeutige Städte)
cluster_1_info = df[df['City'].isin(cluster_1_staedte)][['City', 'Country', 'Population']]

# Doppelte Städte entfernen, falls df_pm25 mehrere Zeilen pro Stadt hat
cluster_1_info = cluster_1_info.drop_duplicates(subset='City').set_index('City')

# Nach Population sortieren (optional)
cluster_1_info = cluster_1_info.sort_values(by='Population', ascending=False)

# Liste der Städte aus Cluster 1 speichern
cluster_1_info.to_csv('data/cluster_1_staedte.csv')

In [ ]:
# Cluster 5-Städte extrahieren
cluster_5_staedte = X_cleaned_final[X_cleaned_final['Cluster'] == 5].index

# Länder- und Populationsdaten holen
cluster_5_info = df[df['City'].isin(cluster_5_staedte)][['City', 'Country', 'Population']]
cluster_5_info = cluster_5_info.drop_duplicates(subset='City').set_index('City')

# Nach Land gruppieren und sortieren
cluster_5_by_country = cluster_5_info.groupby('Country').apply(lambda x: x[['Population']].sort_values(by='Population', ascending=False))

cluster_5_by_country.to_csv('data/cluster_5_staedte.csv')

# Unterschiede Cluster 0 und Cluster 2 herausarbeiten

In [ ]:
# Differenz: Cluster 0 minus Cluster 2
diff_0_2 = (cluster_profiles.loc[0] - cluster_profiles.loc[2]).round(2)
print(diff_0_2)

In [ ]:
# Cluster 0-Städte
cluster_0_staedte = X_cleaned_final[X_cleaned_final['Cluster'] == 0].index
cluster_0_info = df[df['City'].isin(cluster_0_staedte)][['City', 'Country', 'Population']]
cluster_0_info = cluster_0_info.drop_duplicates(subset='City').set_index('City')
cluster_0_info.to_csv('data/cluster_0_staedte.csv')

# Cluster 2-Städte
cluster_2_staedte = X_cleaned_final[X_cleaned_final['Cluster'] == 2].index
cluster_2_info = df[df['City'].isin(cluster_2_staedte)][['City', 'Country', 'Population']]
cluster_2_info = cluster_2_info.drop_duplicates(subset='City').set_index('City')
cluster_2_info.to_csv('data/cluster_2_staedte.csv')

### Unterscheidung Cluster 1 und 5 (beide hohe Schadstoffbelastung)

Cluster 0 und Cluster 2 umfassen Städte mit insgesamt sehr hoher Luftverschmutzung, unterscheiden sich jedoch deutlich in der Art der dominierenden Schadstoffe.

Cluster 0 ist geprägt von extrem hohen Feinstaubwerten (PM2.5 und PM10), während die Konzentrationen gasförmiger Schadstoffe wie Kohlenmonoxid (CO), Stickstoffdioxid (NO₂) und Schwefeldioxid (SO₂) vergleichsweise niedriger ausfallen. Diese Belastung ist typisch für viele schnell wachsende Megastädte, wie sie in China, Indien und angrenzenden Regionen zu finden sind. Häufige Ursachen sind Hausbrand, industrielle Emissionen, ungünstige Wetterlagen und hohe Bevölkerungsdichte.

Cluster 2 hingegen weist sehr hohe Konzentrationen gasförmiger Luftschadstoffe auf – insbesondere CO und SO₂ –, während die Feinstaubwerte zwar erhöht, aber deutlich geringer sind als in Cluster 0. Die Städte in Cluster 2 liegen überwiegend im Iran und in der Türkei, mit vereinzelten Städten in Israel und Südostasien. Die Belastung in diesen Regionen dürfte vor allem auf verkehrs- und energiebedingte Emissionen, veraltete Infrastruktur und industrielle Verbrennungsprozesse zurückzuführen sein.

Die Trennung dieser beiden Cluster ist daher inhaltlich gut nachvollziehbar: Cluster 0 steht für partikelgetriebene Belastung, Cluster 2 für eine gasförmige Emissionsdominanz. Beide stellen gesundheitlich bedenkliche Situationen dar, aber mit unterschiedlichen Quellenprofilen, die jeweils spezifische Gegenmaßnahmen erfordern würden.

# Unterschiede Cluster 0 und Cluster 2 herausarbeiten

In [ ]:
# Differenz: Cluster 3 minus Cluster 4
diff_3_4 = (cluster_profiles.loc[3] - cluster_profiles.loc[4]).round(2)
print(diff_3_4)

In [ ]:
# Cluster 3-Städte
cluster_3_staedte = X_cleaned_final[X_cleaned_final['Cluster'] == 3].index
cluster_3_info = df[df['City'].isin(cluster_3_staedte)][['City', 'Country', 'Population']]
cluster_3_info = cluster_3_info.drop_duplicates(subset='City').set_index('City')
cluster_3_info.to_csv('data/cluster_3_staedte.csv')

# Cluster 4-Städte
cluster_4_staedte = X_cleaned_final[X_cleaned_final['Cluster'] == 4].index
cluster_4_info = df[df['City'].isin(cluster_4_staedte)][['City', 'Country', 'Population']]
cluster_4_info = cluster_4_info.drop_duplicates(subset='City').set_index('City')
cluster_4_info.to_csv('data/cluster_4_staedte.csv')

### Unterscheidung Cluster 3 und 4 (beide mittlere Schadstoffbelastung)

Cluster 3 und Cluster 4 umfassen Städte mit moderater Luftbelastung, unterscheiden sich jedoch in ihrer dominierenden Schadstoffzusammensetzung und regionalen Verteilung.

Cluster 3 vereint Städte mit leicht erhöhter Feinstaubbelastung (PM2.5, PM10) sowie etwas höheren Werten bei CO und SO₂. Diese Belastungskonstellation weist auf gemischte Emissionsquellen wie Hausbrand, Industrie und Verkehr hin. Geografisch liegen die Städte überwiegend in Süd- und Südostasien, Lateinamerika, der Türkei und Südafrika – also Regionen mit teils unvollständiger Filtertechnik und wachsender Urbanisierung.

Cluster 4 dagegen ist geprägt durch erhöhte Ozon- und NO₂-Werte, bei insgesamt geringerer Partikelbelastung. Diese Konstellation ist typisch für Städte mit starker Verkehrsdichte und intensiver Photochemie, etwa in sonnigen, urbanen Küstenregionen oder dicht besiedelten Stadtstaaten. Cluster 4 umfasst hauptsächlich Städte in Südkorea, Taiwan, China, Mexiko und Südeuropa.

Die Trennung dieser beiden Cluster ist daher gut begründet: Cluster 3 steht für eine diffuse, partikelgetriebene Belastung, während Cluster 4 eher für verkehrsbedingte Ozonbildung und Stickstoffoxidbelastung steht – also unterschiedliche urbane Luftbelastungsmodelle mit verschiedenen Ursachen und Lösungsansätzen.

In [ ]:
# Übersichtstabelle erstellen
data = {
    "Cluster": ["1", "5", "4", "3", "2", "0"],
    "Belastungsprofil": [
        "Sehr saubere Luft",
        "Saubere Luft, urban geprägt",
        "Ozon- & NO₂-betonte Belastung",
        "Partikelbetonte Mischung",
        "Gasdominierte Belastung",
        "Extreme Feinstaubbelastung"
    ],
    "Hauptschadstoffe": [
        "keine dominant",
        "O₃, PM2.5",
        "O₃, NO₂",
        "PM2.5, CO, SO₂",
        "CO, SO₂, NO₂",
        "PM2.5, PM10"
    ],
    "Regionale Tendenz": [
        "Australien, Neuseeland, Kanada",
        "Europa, Japan, USA",
        "Südkorea, Taiwan, Südeuropa",
        "Südostasien, Südafrika, Mexiko",
        "Iran, Türkei, Israel",
        "China, Indien, VAE"
    ],
    "Mögliche Ursachen": [
        "Geringe Emissionen, gute Durchlüftung",
        "Verkehr, Photochemie, urbane Dichte",
        "Verkehr, Sonne, hohe Bevölkerungsdichte",
        "Hausbrand, lokale Industrie, gemischte Quellen",
        "Verkehr, fossile Energie, Industrieabgase",
        "Industrie, Hausbrand, Inversion, Urbanisierung"
    ]
}

df_overview = pd.DataFrame(data)

df_overview